In [1]:
import numpy as np
import pandas as pd
from sklearn.linear_model import LinearRegression, BayesianRidge
from sklearn.metrics import r2_score, mean_squared_error
from scipy.stats import norm
from GA_UQ1 import GA_UQ
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

n_runs = 5

def evaluate_methods(X_train, y_train, X_test, y_test):
    # Train FUQ (GA)
    model_fuq = LinearRegression()
    model_fuq.fit(X_train, y_train)
    y_pred_fuq = model_fuq.predict(X_test)
    r2_fuq = r2_score(y_test, y_pred_fuq)
    mse_fuq = mean_squared_error(y_test, y_pred_fuq)
    solution, _, _ = GA_UQ(list(y_pred_fuq),list(y_test))
    a, b = solution
    lower = y_pred_fuq - b
    upper = y_pred_fuq + a
    picp_fuq = np.mean((y_test >= lower) & (y_test <= upper))
    mpiw_fuq = np.mean(upper - lower)

    # Train Bayesian Ridge
    model_bayes = BayesianRidge()
    model_bayes.fit(X_train, y_train)
    y_mean, y_std = model_bayes.predict(X_test, return_std=True)
    z = norm.ppf(0.95)  # 90% prediction interval
    lower_bayes = y_mean - z * y_std
    upper_bayes = y_mean + z * y_std
    r2_bayes = r2_score(y_test, y_mean)
    mse_bayes = mean_squared_error(y_test, y_mean)
    picp_bayes = np.mean((y_test >= lower_bayes) & (y_test <= upper_bayes))
    mpiw_bayes = np.mean(upper_bayes - lower_bayes)

    return {
        "  R2_LinearRegression": r2_fuq,
        "RMSE_LinearRegression": mse_fuq**.5,
        "PICP_FUQ": picp_fuq,
        "MPIW_FUQ": mpiw_fuq,
        "R2_Bayes": r2_bayes,
        "RMSE_Bayes": mse_bayes**.5,
        "PICP_Bayes": picp_bayes,
        "MPIW_Bayes": mpiw_bayes
    }


## Boston Housing

In [2]:
import numpy as np
from sklearn.datasets import fetch_openml
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
import matplotlib.pyplot as plt

# Load the Boston Housing dataset
data = fetch_openml(data_id=531)

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
import numpy as np

X_train, X_test, y_train_full, y_test = train_test_split(data.data, data.target, test_size=0.2, random_state=42)

scaler_X = MinMaxScaler()

X_train_full = scaler_X.fit_transform(X_train)
X_test = scaler_X.transform(X_test)

# Ensure they are NumPy arrays (in case loaded as pandas objects)
X_train_full = np.array(X_train_full)
y_train_full = np.array(y_train_full)
X_test = np.array(X_test)
y_test = np.array(y_test)


In [ ]:
### 1. Noise Addition
noise_levels = [0.0, 0.4, 0.8, 1.0]
noise_results = []
for noise_level in noise_levels:
    metrics = []
    for run in range(n_runs):
        np.random.seed(42 + run)
        noise = np.random.normal(0, noise_level * np.std(y_train_full), size=len(y_train_full))
        y_train_noisy = y_train_full + noise
        metrics.append(evaluate_methods(X_train_full, y_train_noisy, X_test, y_test))
    avg = pd.DataFrame(metrics).mean().to_dict()
    avg['Noise_Level'] = noise_level
    noise_results.append(avg)
df_noise_boston = pd.DataFrame(noise_results)
df_noise_boston

c:\Users\naima\AppData\Local\Programs\Python\Python313\Lib\site-packages\pygad\pygad.py:744: UserWarning: The percentage of genes to mutate (mutation_percent_genes=10) resulted in selecting (0) genes. The number of genes to mutate is set to 1 (mutation_num_genes=1).
If you do not want to mutate any gene, please set mutation_type=None.
  warnings.warn(f"The percentage of genes to mutate (mutation_percent_genes={mutation_percent_genes}) resulted in selecting ({mutation_num_genes}) genes. The number of genes to mutate is set to 1 (mutation_num_genes=1).\nIf you do not want to mutate any gene, please set mutation_type=None.")
c:\Users\naima\AppData\Local\Programs\Python\Python313\Lib\site-packages\pygad\pygad.py:1145: UserWarning: Use the 'save_solutions' parameter with caution as it may cause memory overflow when either the number of generations, number of genes, or number of solutions in population is large.
  warnings.warn("Use the 'save_solutions' parameter with caution as it may cause

,R2_LinearRegression,RMSE_LinearRegression,PICP_FUQ,MPIW_FUQ,R2_Bayes,RMSE_Bayes,PICP_Bayes,MPIW_Bayes,Noise_Level
0,0.668759,4.928602,0.901961,13.348624,0.671251,4.910031,0.941176,17.775635,0.0
1,0.657281,5.012059,0.909804,14.349339,0.661961,4.977901,0.960784,22.505617,0.4
2,0.633005,5.184045,0.921569,15.601803,0.645406,5.096448,0.974510,32.584462,0.8
3,0.616068,5.301037,0.923529,16.210386,0.635415,5.166738,0.984314,38.336647,1.0


In [4]:
### 1. Noise Addition
noise_levels = [0.0, 0.4, 0.8, 1.0]
noise_results = []
for noise_level in noise_levels:
    metrics = []
    for run in range(n_runs):
        np.random.seed(42 + run)
        noise = np.random.normal(0, noise_level * np.std(y_train_full), size=len(y_train_full))
        y_train_noisy = y_train_full + noise
        noise2 = np.random.normal(0, noise_level * np.std(y_test), size=len(y_test))
        y_test_noisy = y_test + noise2
        metrics.append(evaluate_methods(X_train_full, y_train_noisy, X_test, y_test_noisy))
    avg = pd.DataFrame(metrics).mean().to_dict()
    avg['Noise_Level'] = noise_level
    noise_results.append(avg)
df_noise = pd.DataFrame(noise_results)
df_noise

c:\Users\naima\AppData\Local\Programs\Python\Python313\Lib\site-packages\pygad\pygad.py:744: UserWarning: The percentage of genes to mutate (mutation_percent_genes=10) resulted in selecting (0) genes. The number of genes to mutate is set to 1 (mutation_num_genes=1).
If you do not want to mutate any gene, please set mutation_type=None.
  warnings.warn(f"The percentage of genes to mutate (mutation_percent_genes={mutation_percent_genes}) resulted in selecting ({mutation_num_genes}) genes. The number of genes to mutate is set to 1 (mutation_num_genes=1).\nIf you do not want to mutate any gene, please set mutation_type=None.")
c:\Users\naima\AppData\Local\Programs\Python\Python313\Lib\site-packages\pygad\pygad.py:1145: UserWarning: Use the 'save_solutions' parameter with caution as it may cause memory overflow when either the number of generations, number of genes, or number of solutions in population is large.
  warnings.warn("Use the 'save_solutions' parameter with caution as it may cause

,R2_LinearRegression,RMSE_LinearRegression,PICP_FUQ,MPIW_FUQ,R2_Bayes,RMSE_Bayes,PICP_Bayes,MPIW_Bayes,Noise_Level
0,0.668759,4.928602,0.901961,13.350214,0.671251,4.910031,0.941176,17.775635,0.0
1,0.552772,6.133482,0.958824,24.107448,0.557692,6.099618,0.954902,22.505617,0.4
2,0.352215,8.810239,0.966667,38.450648,0.362633,8.738899,0.950980,32.584462,0.8
3,0.267892,10.374542,0.949020,39.987173,0.281682,10.276060,0.943137,38.336647,1.0


In [5]:
### 2. Data Reduction
fractions = [1.0, 0.7, 0.5, 0.2]
reduction_results = []
for frac in fractions:
    metrics = []
    for run in range(n_runs):
        np.random.seed(42 + run)
        size = int(frac * len(X_train_full))
        idx = np.random.choice(len(X_train_full), size=size, replace=False)
        metrics.append(evaluate_methods(X_train_full[idx], y_train_full[idx], X_test, y_test))
    avg = pd.DataFrame(metrics).mean().to_dict()
    avg['Train_Fraction'] = frac
    reduction_results.append(avg)
df_reduction = pd.DataFrame(reduction_results)
df_reduction

c:\Users\naima\AppData\Local\Programs\Python\Python313\Lib\site-packages\pygad\pygad.py:744: UserWarning: The percentage of genes to mutate (mutation_percent_genes=10) resulted in selecting (0) genes. The number of genes to mutate is set to 1 (mutation_num_genes=1).
If you do not want to mutate any gene, please set mutation_type=None.
  warnings.warn(f"The percentage of genes to mutate (mutation_percent_genes={mutation_percent_genes}) resulted in selecting ({mutation_num_genes}) genes. The number of genes to mutate is set to 1 (mutation_num_genes=1).\nIf you do not want to mutate any gene, please set mutation_type=None.")
c:\Users\naima\AppData\Local\Programs\Python\Python313\Lib\site-packages\pygad\pygad.py:1145: UserWarning: Use the 'save_solutions' parameter with caution as it may cause memory overflow when either the number of generations, number of genes, or number of solutions in population is large.
  warnings.warn("Use the 'save_solutions' parameter with caution as it may cause

,R2_LinearRegression,RMSE_LinearRegression,PICP_FUQ,MPIW_FUQ,R2_Bayes,RMSE_Bayes,PICP_Bayes,MPIW_Bayes,Train_Fraction
0,0.668759,4.928602,0.901961,13.341483,0.671251,4.910031,0.941176,17.775635,1.0
1,0.661777,4.977846,0.913725,14.390384,0.666428,4.943651,0.941176,18.219344,0.7
2,0.647334,5.084193,0.900000,13.679323,0.653659,5.038628,0.949020,19.041606,0.5
3,0.612889,5.308011,0.898039,15.174190,0.654130,5.032529,0.966667,24.202557,0.2


In [6]:
### 3. Output Coarsening
from sklearn.preprocessing import KBinsDiscretizer

# --- Coarsening setup ---
coarsening_levels = [1.0, 0.5, .25]  # 100%, 50%, 25% precision
n_runs = 5
coarsening_results = []

# --- Quantile coarsening ---
def quantile_coarsening_with_midpoints(y, n_bins):
    kbin = KBinsDiscretizer(n_bins=n_bins, encode='ordinal', strategy='quantile')
    y_binned = kbin.fit_transform(y.reshape(-1, 1)).ravel()
    bin_edges = kbin.bin_edges_[0]
    # Replace each bin label with the midpoint of that bin
    y_midpoints = np.array([ (bin_edges[int(b)] + bin_edges[int(b)+1]) / 2 for b in y_binned ])
    return y_midpoints

# --- Run experiments ---
for c in coarsening_levels:
    metrics = []
    bins = max(2, int(10 * c))  # Keep bins reasonable
    for run in range(n_runs):
        np.random.seed(42 + run)
        y_train_coarse = quantile_coarsening_with_midpoints(y_train_full, n_bins=bins)
        y_test_coarse = quantile_coarsening_with_midpoints(y_test, n_bins=bins)
        metrics.append(evaluate_methods(X_train_full, y_train_coarse, X_test, y_test_coarse))
    avg = pd.DataFrame(metrics).mean().to_dict()
    avg['Coarsening_Factor'] = c
    coarsening_results.append(avg)

# --- Results ---
df_coarse = pd.DataFrame(coarsening_results)
df_coarse

c:\Users\naima\AppData\Local\Programs\Python\Python313\Lib\site-packages\pygad\pygad.py:744: UserWarning: The percentage of genes to mutate (mutation_percent_genes=10) resulted in selecting (0) genes. The number of genes to mutate is set to 1 (mutation_num_genes=1).
If you do not want to mutate any gene, please set mutation_type=None.
  warnings.warn(f"The percentage of genes to mutate (mutation_percent_genes={mutation_percent_genes}) resulted in selecting ({mutation_num_genes}) genes. The number of genes to mutate is set to 1 (mutation_num_genes=1).\nIf you do not want to mutate any gene, please set mutation_type=None.")
c:\Users\naima\AppData\Local\Programs\Python\Python313\Lib\site-packages\pygad\pygad.py:1145: UserWarning: Use the 'save_solutions' parameter with caution as it may cause memory overflow when either the number of generations, number of genes, or number of solutions in population is large.
  warnings.warn("Use the 'save_solutions' parameter with caution as it may cause

,R2_LinearRegression,RMSE_LinearRegression,PICP_FUQ,MPIW_FUQ,R2_Bayes,RMSE_Bayes,PICP_Bayes,MPIW_Bayes,Coarsening_Factor
0,0.681665,4.715128,0.901961,14.107712,0.685049,4.690002,0.901961,16.999014,1.00
1,0.600060,5.869346,0.911765,19.514131,0.602536,5.851148,0.911765,18.486368,0.50
2,0.536144,7.662035,1.000000,38.749288,0.542349,7.610619,0.941176,27.677966,0.25


In [7]:
print("\n=== Noise Addition ===")
print(df_noise)

print("\n=== Data Reduction ===")
print(df_reduction)

print("\n=== Output Coarsening ===")
print(df_coarse)



=== Noise Addition ===
     R2_LinearRegression  RMSE_LinearRegression  PICP_FUQ   MPIW_FUQ  \
0               0.668759               4.928602  0.901961  13.350214   
1               0.552772               6.133482  0.958824  24.107448   
2               0.352215               8.810239  0.966667  38.450648   
3               0.267892              10.374542  0.949020  39.987173   

   R2_Bayes  RMSE_Bayes  PICP_Bayes  MPIW_Bayes  Noise_Level  
0  0.671251    4.910031    0.941176   17.775635          0.0  
1  0.557692    6.099618    0.954902   22.505617          0.4  
2  0.362633    8.738899    0.950980   32.584462          0.8  
3  0.281682   10.276060    0.943137   38.336647          1.0  

=== Data Reduction ===
     R2_LinearRegression  RMSE_LinearRegression  PICP_FUQ   MPIW_FUQ  \
0               0.668759               4.928602  0.901961  13.341483   
1               0.661777               4.977846  0.913725  14.390384   
2               0.647334               5.084193  0.900000  1

## California Housing

In [8]:
from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
import numpy as np

# Fetch the California housing dataset
data = fetch_california_housing()

X_train, X_test, y_train_full, y_test = train_test_split(data.data, data.target, test_size=0.2, random_state=42)
scaler_X = MinMaxScaler()

X_train_full = scaler_X.fit_transform(X_train)
X_test = scaler_X.transform(X_test)

# Ensure they are NumPy arrays (in case loaded as pandas objects)
X_train_full = np.array(X_train_full)
y_train_full = np.array(y_train_full)
X_test = np.array(X_test)
y_test = np.array(y_test)

In [9]:
### 1. Noise Addition
noise_levels = [0.0, 0.4, .8,1]
noise_results = []
for noise_level in noise_levels:
    metrics = []
    for run in range(n_runs):
        np.random.seed(42 + run)
        noise = np.random.normal(0, noise_level * np.std(y_train_full), size=len(y_train_full))
        y_train_noisy = y_train_full + noise
        metrics.append(evaluate_methods(X_train_full, y_train_noisy, X_test, y_test))
    avg = pd.DataFrame(metrics).mean().to_dict()
    avg['Noise_Level'] = noise_level
    noise_results.append(avg)
df_noise = pd.DataFrame(noise_results)
df_noise

c:\Users\naima\AppData\Local\Programs\Python\Python313\Lib\site-packages\pygad\pygad.py:744: UserWarning: The percentage of genes to mutate (mutation_percent_genes=10) resulted in selecting (0) genes. The number of genes to mutate is set to 1 (mutation_num_genes=1).
If you do not want to mutate any gene, please set mutation_type=None.
  warnings.warn(f"The percentage of genes to mutate (mutation_percent_genes={mutation_percent_genes}) resulted in selecting ({mutation_num_genes}) genes. The number of genes to mutate is set to 1 (mutation_num_genes=1).\nIf you do not want to mutate any gene, please set mutation_type=None.")
c:\Users\naima\AppData\Local\Programs\Python\Python313\Lib\site-packages\pygad\pygad.py:1145: UserWarning: Use the 'save_solutions' parameter with caution as it may cause memory overflow when either the number of generations, number of genes, or number of solutions in population is large.
  warnings.warn("Use the 'save_solutions' parameter with caution as it may cause

,R2_LinearRegression,RMSE_LinearRegression,PICP_FUQ,MPIW_FUQ,R2_Bayes,RMSE_Bayes,PICP_Bayes,MPIW_Bayes,Noise_Level
0,0.575788,0.745581,0.947384,3.031536,0.576123,0.745287,0.916424,2.381184,0.0
1,0.576090,0.745315,0.946560,3.025555,0.576560,0.744902,0.941473,2.836180,0.4
2,0.576224,0.745198,0.946657,3.030324,0.577094,0.744432,0.971609,3.888036,0.8
3,0.576227,0.745195,0.946415,3.031751,0.577394,0.744169,0.983140,4.518330,1.0


In [10]:
### 2. Data Reduction
fractions = [1.0, 0.5, 0.1, 0.05]
reduction_results = []
for frac in fractions:
    metrics = []
    for run in range(n_runs):
        np.random.seed(42 + run)
        size = int(frac * len(X_train_full))
        idx = np.random.choice(len(X_train_full), size=size, replace=False)
        metrics.append(evaluate_methods(X_train_full[idx], y_train_full[idx], X_test, y_test))
    avg = pd.DataFrame(metrics).mean().to_dict()
    avg['Train_Fraction'] = frac
    reduction_results.append(avg)
df_reduction = pd.DataFrame(reduction_results)
df_reduction

c:\Users\naima\AppData\Local\Programs\Python\Python313\Lib\site-packages\pygad\pygad.py:744: UserWarning: The percentage of genes to mutate (mutation_percent_genes=10) resulted in selecting (0) genes. The number of genes to mutate is set to 1 (mutation_num_genes=1).
If you do not want to mutate any gene, please set mutation_type=None.
  warnings.warn(f"The percentage of genes to mutate (mutation_percent_genes={mutation_percent_genes}) resulted in selecting ({mutation_num_genes}) genes. The number of genes to mutate is set to 1 (mutation_num_genes=1).\nIf you do not want to mutate any gene, please set mutation_type=None.")
c:\Users\naima\AppData\Local\Programs\Python\Python313\Lib\site-packages\pygad\pygad.py:1145: UserWarning: Use the 'save_solutions' parameter with caution as it may cause memory overflow when either the number of generations, number of genes, or number of solutions in population is large.
  warnings.warn("Use the 'save_solutions' parameter with caution as it may cause

,R2_LinearRegression,RMSE_LinearRegression,PICP_FUQ,MPIW_FUQ,R2_Bayes,RMSE_Bayes,PICP_Bayes,MPIW_Bayes,Train_Fraction
0,0.575788,0.745581,0.947384,3.029214,0.576123,0.745287,0.916424,2.381184,1.00
1,0.578402,0.743274,0.946705,3.032961,0.578845,0.742883,0.919089,2.403091,0.50
2,0.532463,0.781922,0.947917,2.966969,0.539888,0.775799,0.925969,2.444278,0.10
3,0.514607,0.795948,0.948643,2.967982,0.537139,0.777422,0.931928,2.575445,0.05


In [11]:
### 3. Output Coarsening
from sklearn.preprocessing import KBinsDiscretizer

# --- Coarsening setup ---
coarsening_levels = [1.0, 0.5, 0.25]  # 100%, 50%, 25% precision
n_runs = 5
coarsening_results = []

# --- Quantile coarsening ---
def quantile_coarsening_with_midpoints(y, n_bins):
    kbin = KBinsDiscretizer(n_bins=n_bins, encode='ordinal', strategy='quantile')
    y_binned = kbin.fit_transform(y.reshape(-1, 1)).ravel()
    bin_edges = kbin.bin_edges_[0]
    # Replace each bin label with the midpoint of that bin
    y_midpoints = np.array([ (bin_edges[int(b)] + bin_edges[int(b)+1]) / 2 for b in y_binned ])
    return y_midpoints

# --- Run experiments ---
for c in coarsening_levels:
    metrics = []
    bins = max(2, int(10 * c))  # Keep bins reasonable
    for run in range(n_runs):
        np.random.seed(42 + run)
        y_train_coarse = quantile_coarsening_with_midpoints(y_train_full, n_bins=bins)
        y_test_coarse = quantile_coarsening_with_midpoints(y_test, n_bins=bins)
        metrics.append(evaluate_methods(X_train_full, y_train_coarse, X_test, y_test_coarse))
    avg = pd.DataFrame(metrics).mean().to_dict()
    avg['Coarsening_Factor'] = c
    coarsening_results.append(avg)

# --- Results ---
df_coarse = pd.DataFrame(coarsening_results)
df_coarse

c:\Users\naima\AppData\Local\Programs\Python\Python313\Lib\site-packages\pygad\pygad.py:744: UserWarning: The percentage of genes to mutate (mutation_percent_genes=10) resulted in selecting (0) genes. The number of genes to mutate is set to 1 (mutation_num_genes=1).
If you do not want to mutate any gene, please set mutation_type=None.
  warnings.warn(f"The percentage of genes to mutate (mutation_percent_genes={mutation_percent_genes}) resulted in selecting ({mutation_num_genes}) genes. The number of genes to mutate is set to 1 (mutation_num_genes=1).\nIf you do not want to mutate any gene, please set mutation_type=None.")
c:\Users\naima\AppData\Local\Programs\Python\Python313\Lib\site-packages\pygad\pygad.py:1145: UserWarning: Use the 'save_solutions' parameter with caution as it may cause memory overflow when either the number of generations, number of genes, or number of solutions in population is large.
  warnings.warn("Use the 'save_solutions' parameter with caution as it may cause

,R2_LinearRegression,RMSE_LinearRegression,PICP_FUQ,MPIW_FUQ,R2_Bayes,RMSE_Bayes,PICP_Bayes,MPIW_Bayes,Coarsening_Factor
0,0.571778,0.725395,0.950339,3.194780,0.572125,0.725101,0.909641,2.324863,1.00
1,0.536272,0.770071,0.962016,3.628379,0.536643,0.769764,0.897771,2.496818,0.50
2,0.393354,0.944390,0.997335,4.994745,0.393912,0.943955,0.947190,3.081681,0.25


In [12]:
print("\n=== Noise Addition ===")
print(df_noise)

print("\n=== Data Reduction ===")
print(df_reduction)

print("\n=== Output Coarsening ===")
print(df_coarse)


=== Noise Addition ===
     R2_LinearRegression  RMSE_LinearRegression  PICP_FUQ  MPIW_FUQ  R2_Bayes  \
0               0.575788               0.745581  0.947384  3.031536  0.576123   
1               0.576090               0.745315  0.946560  3.025555  0.576560   
2               0.576224               0.745198  0.946657  3.030324  0.577094   
3               0.576227               0.745195  0.946415  3.031751  0.577394   

   RMSE_Bayes  PICP_Bayes  MPIW_Bayes  Noise_Level  
0    0.745287    0.916424    2.381184          0.0  
1    0.744902    0.941473    2.836180          0.4  
2    0.744432    0.971609    3.888036          0.8  
3    0.744169    0.983140    4.518330          1.0  

=== Data Reduction ===
     R2_LinearRegression  RMSE_LinearRegression  PICP_FUQ  MPIW_FUQ  R2_Bayes  \
0               0.575788               0.745581  0.947384  3.029214  0.576123   
1               0.578402               0.743274  0.946705  3.032961  0.578845   
2               0.532463              

# Auto

In [13]:
import numpy as np
import pandas as pd
from sklearn.datasets import fetch_openml
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score

# Load the Automobile MPG dataset from OpenML
#mpg = fetch_openml(name='autompg',version=3)
mpg = fetch_openml(data_id=42372)
# Convert to DataFrame
X = pd.DataFrame(mpg.data, columns=mpg.feature_names)
y = pd.DataFrame(mpg.target)

from sklearn.preprocessing import MinMaxScaler
scaler_X = MinMaxScaler()
X_train_scaled = scaler_X.fit_transform(X)
X=X_train_scaled
y=np.array(y)

In [14]:
from sklearn.model_selection import KFold
from sklearn.metrics import r2_score, mean_squared_error
from scipy.stats import norm
import numpy as np

def train_and_report( X_train, y_train):
    kfold = KFold(n_splits=10, shuffle=True, random_state=42)

    model_fuq=LinearRegression()
    model_bayes=BayesianRidge()
    predictions_fuq = []
    predictions_bayes = []
    actual_values = []
    std_bayes_all = []

    for train_index, test_index in kfold.split(X_train):
        X_train_fold, X_test_fold = X_train[train_index], X_train[test_index]
        y_train_fold, y_test_fold = y_train[train_index], y_train[test_index]

        # Train FUQ model (e.g., Linear Regression)
        model_fuq.fit(X_train_fold, y_train_fold)
        y_pred_fuq = model_fuq.predict(X_test_fold)
        predictions_fuq.extend(y_pred_fuq)

        # Train Bayesian model
        model_bayes.fit(X_train_fold, y_train_fold)
        y_pred_bayes, y_std_bayes = model_bayes.predict(X_test_fold, return_std=True)
        predictions_bayes.extend(y_pred_bayes)
        std_bayes_all.extend(y_std_bayes)

        actual_values.extend(y_test_fold)

    predictions_fuq = np.array(predictions_fuq)
    predictions_bayes = np.array(predictions_bayes)
    actual_values = np.array(actual_values)
    std_bayes_all = np.array(std_bayes_all)

    # FUQ Evaluation
    r2_fuq = r2_score(actual_values, predictions_fuq)
    mse_fuq = mean_squared_error(actual_values, predictions_fuq)

    solution, _, _ = GA_UQ(predictions_fuq, actual_values)
    a_ga, b_ga = solution
    lower_ga = predictions_fuq - b_ga
    upper_ga = predictions_fuq + a_ga
    picp_ga = np.mean((actual_values >= lower_ga) & (actual_values <= upper_ga))
    mpiw_ga = np.mean(upper_ga - lower_ga)

    # Bayesian Evaluation
    r2_bayes = r2_score(actual_values, predictions_bayes)
    mse_bayes = mean_squared_error(actual_values, predictions_bayes)
    z = norm.ppf(0.95)  # for ~90% PI
    lower_bayes = predictions_bayes - z * std_bayes_all
    upper_bayes = predictions_bayes + z * std_bayes_all
    picp_bayes = np.mean((actual_values >= lower_bayes) & (actual_values <= upper_bayes))
    mpiw_bayes = np.mean(upper_bayes - lower_bayes)

    result = {
       
        "R2_FUQ": r2_fuq,
        "RMSE_FUQ": mse_fuq**.5,
        "PICP_FUQ": picp_ga,
        "MPIW_FUQ": mpiw_ga,
        "a_GA": a_ga,
        "b_GA": b_ga,
        "R2_Bayes": r2_bayes,
        "RMSE_Bayes": mse_bayes**.5,
        "PICP_Bayes": picp_bayes,
        "MPIW_Bayes": mpiw_bayes,
        
    }

    return result


In [15]:

# Using Linear Regression model with reduced data fractions

results=[]


results = []
for frac in [1.0, 0.7,0.5, 0.2]:
    for run in range(n_runs):
        np.random.seed(41 + run)
        size = int(frac * len(X))
        idx = np.random.choice(len(X), size=size, replace=False)
        X_reduced = X[idx]
        y_reduced = y[idx]

        result = train_and_report( X_reduced, y_reduced)
        result['Train_Fraction']=frac
        results.append(result)

# Convert results to DataFrame for analysis
df_reduction = pd.DataFrame(results)
df_reduction


c:\Users\naima\AppData\Local\Programs\Python\Python313\Lib\site-packages\sklearn\utils\validation.py:1408: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
c:\Users\naima\AppData\Local\Programs\Python\Python313\Lib\site-packages\sklearn\utils\validation.py:1408: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
c:\Users\naima\AppData\Local\Programs\Python\Python313\Lib\site-packages\sklearn\utils\validation.py:1408: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
c:\Users\naima\AppData\Local\Programs\Python\Python313\Lib\site-packages\sklearn\utils\validation.py:1408: DataC

,R2_FUQ,RMSE_FUQ,PICP_FUQ,MPIW_FUQ,a_GA,b_GA,R2_Bayes,RMSE_Bayes,PICP_Bayes,MPIW_Bayes,Train_Fraction
0,0.798503,3.499070,0.951531,14.124755,7.499214,6.625541,0.798669,3.497627,0.407187,11.979996,1.0
1,0.803665,3.453960,0.951531,14.159933,7.504662,6.655270,0.803627,3.454299,0.408105,11.989521,1.0
2,0.798379,3.500150,0.951531,14.301821,7.511362,6.790459,0.798464,3.499413,0.408430,11.981769,1.0
3,0.802388,3.465175,0.954082,14.151446,7.315211,6.836236,0.802317,3.465801,0.408059,11.989121,1.0
4,0.802087,3.467811,0.956633,14.188988,7.394477,6.794512,0.802161,3.467166,0.408319,11.986289,1.0
5,0.810340,3.471934,0.959854,14.310227,7.509033,6.801194,0.810302,3.472286,0.404177,12.117650,0.7
6,0.807342,3.318592,0.974453,14.511093,7.303450,7.207643,0.807697,3.315534,0.401553,11.528514,0.7
7,0.805350,3.431656,0.963504,14.304579,6.998500,7.306079,0.805766,3.427992,0.407600,12.006968,0.7
8,0.799584,3.340694,0.963504,13.844371,7.090193,6.754178,0.799304,3.343020,0.417763,11.768110,0.7
9,0.788541,3.574151,0.956204,14.699522,7.784385,6.915137,0.788937,3.570799,0.422186,12.416974,0.7


In [16]:
### 1. Noise Addition
noise_levels = [0.0, 0.4, .8,1]
y=np.ravel(y)
noise_results = []
for noise_level in noise_levels:
    metrics = []
    for run in range(n_runs):
        np.random.seed(42 + run)
        noise = np.random.normal(0, noise_level * np.std(y), size=len(y))
        y_train_noisy = y + noise
        metrics.append(train_and_report(X, y_train_noisy))
    avg = pd.DataFrame(metrics).mean().to_dict()
    avg['Noise_Level'] = noise_level
    noise_results.append(avg)
df_noise = pd.DataFrame(noise_results)
df_noise

c:\Users\naima\AppData\Local\Programs\Python\Python313\Lib\site-packages\pygad\pygad.py:744: UserWarning: The percentage of genes to mutate (mutation_percent_genes=10) resulted in selecting (0) genes. The number of genes to mutate is set to 1 (mutation_num_genes=1).
If you do not want to mutate any gene, please set mutation_type=None.
  warnings.warn(f"The percentage of genes to mutate (mutation_percent_genes={mutation_percent_genes}) resulted in selecting ({mutation_num_genes}) genes. The number of genes to mutate is set to 1 (mutation_num_genes=1).\nIf you do not want to mutate any gene, please set mutation_type=None.")
c:\Users\naima\AppData\Local\Programs\Python\Python313\Lib\site-packages\pygad\pygad.py:1145: UserWarning: Use the 'save_solutions' parameter with caution as it may cause memory overflow when either the number of generations, number of genes, or number of solutions in population is large.
  warnings.warn("Use the 'save_solutions' parameter with caution as it may cause

,R2_FUQ,RMSE_FUQ,PICP_FUQ,MPIW_FUQ,a_GA,b_GA,R2_Bayes,RMSE_Bayes,PICP_Bayes,MPIW_Bayes,Noise_Level
0,0.802567,3.463610,0.951531,13.928465,7.253025,6.675439,0.802520,3.464020,0.931122,11.990740,0.0
1,0.693607,4.672574,0.964286,19.927870,10.029788,9.898082,0.693549,4.673019,0.914796,16.173716,0.4
2,0.495753,7.120705,0.967347,31.267247,15.426931,15.840316,0.495834,7.120182,0.910714,24.541178,0.8
3,0.408880,8.501449,0.966327,37.082853,18.381972,18.700881,0.409180,8.499386,0.904592,29.214450,1.0


In [17]:
coarsening_levels = [1.0, 0.5, 0.25]  # 100%, 50%, 25% precision
n_runs = 5
coarsening_results = []

# --- Quantile coarsening ---
def quantile_coarsening_with_midpoints(y, n_bins):
    kbin = KBinsDiscretizer(n_bins=n_bins, encode='ordinal', strategy='quantile')
    y_binned = kbin.fit_transform(y.reshape(-1, 1)).ravel()
    bin_edges = kbin.bin_edges_[0]
    # Replace each bin label with the midpoint of that bin
    y_midpoints = np.array([ (bin_edges[int(b)] + bin_edges[int(b)+1]) / 2 for b in y_binned ])
    return y_midpoints

# --- Run experiments ---
for c in coarsening_levels:
    metrics = []
    bins = max(2, int(10 * c))  # Keep bins reasonable
    for run in range(n_runs):
        np.random.seed(42 + run)
        y_train_coarse = quantile_coarsening_with_midpoints(y, n_bins=bins)
        metrics.append(train_and_report(X, y_train_coarse))
    avg = pd.DataFrame(metrics).mean().to_dict()
    avg['Coarsening_Factor'] = c
    coarsening_results.append(avg)

# --- Results ---
df_coarse = pd.DataFrame(coarsening_results)
df_coarse

c:\Users\naima\AppData\Local\Programs\Python\Python313\Lib\site-packages\pygad\pygad.py:744: UserWarning: The percentage of genes to mutate (mutation_percent_genes=10) resulted in selecting (0) genes. The number of genes to mutate is set to 1 (mutation_num_genes=1).
If you do not want to mutate any gene, please set mutation_type=None.
  warnings.warn(f"The percentage of genes to mutate (mutation_percent_genes={mutation_percent_genes}) resulted in selecting ({mutation_num_genes}) genes. The number of genes to mutate is set to 1 (mutation_num_genes=1).\nIf you do not want to mutate any gene, please set mutation_type=None.")
c:\Users\naima\AppData\Local\Programs\Python\Python313\Lib\site-packages\pygad\pygad.py:1145: UserWarning: Use the 'save_solutions' parameter with caution as it may cause memory overflow when either the number of generations, number of genes, or number of solutions in population is large.
  warnings.warn("Use the 'save_solutions' parameter with caution as it may cause

,R2_FUQ,RMSE_FUQ,PICP_FUQ,MPIW_FUQ,a_GA,b_GA,R2_Bayes,RMSE_Bayes,PICP_Bayes,MPIW_Bayes,Coarsening_Factor
0,0.782288,3.831191,0.969388,16.75895,8.158087,8.600863,0.782107,3.832783,0.908163,13.267847,1.00
1,0.775583,4.237978,0.989796,20.76616,10.460933,10.305227,0.775576,4.238047,0.928571,14.703536,0.50
2,0.624167,5.762690,0.951531,27.01901,15.219143,11.799867,0.623854,5.765089,0.905612,19.995500,0.25
